In [1]:
from src.crypto import CryptoCatalog
cc = CryptoCatalog()


In [5]:
cc.coins[0]

<Coin id:`01coin` symbol:`ZOC` name:`01coin`>

In [2]:
import sys
sys.executable

'C:\\Users\\muril\\.virtualenvs\\insta-post-generator-N5ODM5nx\\Scripts\\python.exe'